# NFT Wash Trading

Here we can build a lab notebook cell by cell.

The first task is to get us both on the same page on what data we'll be using for the project, and then we can work on loading it in and working with it.

## Data import

In [1]:
##Data import code here.
import os
import pandas as pd
cwd = os.getcwd()
bored_ape_location = cwd + '/data/bored_ape.csv'

In [2]:
#Load in the csv
bored_ape = pd.read_csv(bored_ape_location, low_memory=False)

In [3]:
#Infers some column data types to float and int to save some space
bored_ape_converted = bored_ape.infer_objects()

In [4]:
bored_ape_converted[['winner_account_address','winner_account_user_username','seller_user_username', 'seller_address']]

,winner_account_address,winner_account_user_username,seller_user_username,seller_address
0,0x05a9d39e168fd7aca02970be894aea848d85bd33,jrb1013,NaN,0x0141fdeaea923168899a134f2caf724f89a228aa
1,0x053c07c8c7d6cb9b6e78c9df3436cfb0fd72b1de,TP777,ApeFatherVault,0x1edcccddb214cd3c20613708d9c9ead9cd2476db
2,0x3d216c627351e53669f138fd6e1e3e1f31b5dff0,Moo_0x3d,PVNKS-Vault,0x8323cc95c6fc88c832086e38869cfe1d834a4980
3,0x62261bae83431db16d01589256e9b39e0d46079e,allidoiswinwinwin,0xed4701,0xed47015bb8080b9399f9d0ddfc427b9cee2caab1
4,0xd8e749e457fcd1918c9a589bfaa87db9f8e154d6,NaN,NaN,0xe399b5a5fa209b96ceac7c05cf301f2968eebbbd
...,...,...,...,...
26517,0x09238a2aa20600e9773990677d17fef91e688523,shariq911,6942069,0x9795a9baac2f17e7c7f11686bc6bc2afc3d84612
26518,0x09238a2aa20600e9773990677d17fef91e688523,shariq911,1sadboi1,0x3d6a89c8751a45dd577a4c1f3b34e71c58236193
26519,0x09238a2aa20600e9773990677d17fef91e688523,shariq911,1sadboi1,0x3d6a89c8751a45dd577a4c1f3b34e71c58236193
26520,0x0b742783bfac8d4b6d332e5d1b63f433fcd8c0a0,Cryptobeanz_Deployer,madeinmanc,0x88be3fa60ede9f532af10aba5690dfc254db929b


Usernames that are listed as NaN are unnamed on OpenSea but still have a wallet attached.

In [5]:
#Build a simple request on etherscan to scale up for each seller wallet in opensea data
import requests
#Arbitrary address for testing, when this is in a loop it should read from the panda/csv
address = '0x8323cc95c6fc88c832086e38869cfe1d834a4980'
payload = {'module':'account', 
           'action':'txlist', 
           'address':address,
           'startblock':'0',
           'endblock':'99999999',
           'page':'0',
           'offset':'0',
           'sort':'asc',
           'apikey':'5PN91CD3CXHJBPY4U3ZPCG86ZHWJUUZHBF'
          }
r = requests.get('https://api.etherscan.io/api', params=payload)
#Create a dict to store all the transactions
wallet_dict = {}
all_wallet_dict = {}

In [6]:
for transaction in r.json()['result']:
    if transaction['from'] == address:
        wallet_dict[transaction['to']] = 'sent'
    else:
        wallet_dict[transaction['from']] = 'recieved'
all_wallet_dict[address] = wallet_dict

In [7]:
#Shows the result, a dict with the key being the wallet address
#and the value being all transacations + type
print(all_wallet_dict)

{'0x8323cc95c6fc88c832086e38869cfe1d834a4980': {'0xc3fbc3f485f0d9b0bd21b13a4aaa8340160156cb': 'recieved', '0xa5409ec958c83c3f309868babaca7c86dcb077c1': 'sent', '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623': 'sent', '0xedd3ee98fd6050d104ccf983cd13948557057a8f': 'sent', '0xd5f76a01d4ebe570a9252f1a8459b44fe8f4b5bd': 'sent', '0xfc007068c862e69213dc7aa817063b1803d4e941': 'sent', '0x45d8f7db9b437efbc74ba6a945a81aaf62dceda7': 'sent', '0x1b829b926a14634d36625e60165c0770c09d02b2': 'sent', '0xc2c747e0f7004f9e8817db2ca4997657a7746928': 'sent', '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b': 'sent', '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d': 'sent', '0x1b811fab3618e727d5d38d7e6338262ca372a3ca': 'sent', '0xfcc617a026bc7614db0debc00e72ea37d9ab9317': 'sent', '0x35a5add062f5d164bfecc8b5f2e5da10a09872e9': 'sent', '0xcde1efe3c576c900a257687aa95b2efad7150227': 'sent', '0x91bbc6dcfef7a30505106bf887528218cbda3f83': 'sent', '0xdbc410bd7a9a83c3cbe501a26cabada4a60826e2': 'sent', '0xb80a6253ebbdd2f72fe5b6fd0f8

In [8]:
all_wallet_list = []
for wallet_id in bored_ape_converted['winner_account_address']:
    all_wallet_list.append(wallet_id)
#Remove duplicates
all_wallet_list = set(all_wallet_list)
#all_wallet_list

# Dont know what the first nan value is but heres where I left off for the day.

## Analysis

I'll look at the paper again and determine a similar structure of analysis. Market by market, method by method.

In [9]:
##Analysis code here

## Results

In [10]:
## Hopefully some interesting results here